# Creating an R web service with the Azure Machine Learning services
This notebook shows how to write and deploy a scoring webservice with Azure Machine Learning Services (AMLS) that runs R code. Note that we are not using Azure Machine Learning Studio here. Also, this shows how to run an R workload in the context of Python. It does not use any native support of R - which may come in future (or might be already there, depending on when you read this article).

The underlying scenario is a **scoring** scenario. If your scenario is rather compute-intensive **batch computation**, you might rather wanna use the notebook [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.ipynb) as base and adopt some of the ideas contained in this notebook (eg. using rpy2). Similarly, for creating **pipelines**, you can use the samples [here](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/machine-learning-pipelines) as base.

# Requirements

Before you can start, you need

* an Azure Machine Learning services account in Azure.
* If deployed to AKS: an AKS cluster in Azure Machine Learning services.

For brevity reasons, I do not go into details here. There is numerous examples on creating an AKS cluster for Azure Machine Learning services. The easiest way to create one is to create it through the Azure Machine Learning Workspace in Azure Portal. Besides, there are examples [here](https://github.com/Azure/MachineLearningNotebooks) as well as the option to create an AKS through the [Azure ML CLI extension](https://docs.microsoft.com/de-de/azure/machine-learning/service/reference-azure-machine-learning-cli). 

# Get workspace

In [ ]:
# note: this instantiates the workspace object directly.
#       if you have a config file you also use that to get the workspace.
import os
from azureml.core.workspace import Workspace

subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = "azureml-spike-rg"
workspace_name = "OurWorkspace"

workspace = Workspace(subscription_id=subscription_id,
                      resource_group=resource_group,
                      workspace_name=workspace_name)

# Create/train the model

See the included `create_model.r` file so see how a model could be created.

# Register the model
Register an existing trained model, add description and tags.

In [ ]:
from azureml.core.model import Model
model = Model.register(model_path = "model.RData",
                       model_name = "model.RData",
                       tags = {'area': "samples", 'type': "regression"},
                       description = "A simple linear regression model to show the usage of R in Azure Machine Learning Services.",
                       workspace = workspace)

print(model.name, model.description, model.version)

# Create an image
Create an image using
- the registered model (created and registered above),
- a script that will load and run the model and
- all dependencies required by the codes.

In [ ]:
%%writefile score.py
import traceback
import json
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector
import rpy2.rinterface
import rpy2.robjects as robjects

def init():
    # only needed if model data collection is used
    # note: the example here uses a single model data collector. you can however use multiple ones if needed.
    global modeldata_collector
    modeldata_collector = ModelDataCollector("model.RData", identifier="predictions", feature_names=["x", "y"])
    
    # note: this function is run whenever the container is started
    # init rpy2
    rpy2.rinterface.initr()
    # load model
    model_path = Model.get_model_path('model.RData')
    robjects.r("load('{model_path}')".format(model_path=model_path))
    # run init() function in R (if exists)
    robjects.r("if (exists('init', mode='function')) { init() }")

def run(input_json_string): 
    # note: this function is run whenever a scoring request has been received
    try:
        result_vector = robjects.r(
                "run('{input_json_string}')".format(input_json_string=input_json_string)
            )
        if len(result_vector) > 0:
            try:
                # get prediction result              
                prediction_result_json = json.loads(result_vector[0])
                
                # log prediction (only needed if model data is collected)
                input_x = json.loads(input_json_string)["x"]
                prediction_y = prediction_result_json["y"]
                modeldata_collector.collect([input_x, prediction_y])
                
                # return prediction result
                return prediction_result_json
            except ValueError:
                return {"message": result_vector }

    except Exception as e:
        return {           
            "error": repr(e),
            "traceback": traceback.format_exc()
        }

Define the dependencies required by the scoring script to run.

- rpy2 is a package to run R from Python
- mro-base is Microsoft's R distribution
- r-jsonlite and r-essentials are two conda packages which add the jsonlite package and a few other common packages to our R
- azureml-monitoring is needed if you want to use AzureML's model data collection feature

If you need more packages, you can specify them with the conda_packages parameter below - which should be the preferred method.

However, if your packages are not available on conda, you can also tell the image_configuration() function below to add some local files or folders to the image (parameter: dependencies) and then install the packages manually by specifying some Docker commands in a file passed to the docker_file parameter. (The files/folders specified by dependencies parameter will be put into /var/azureml-app. The docker_file is run before the conda_packages. Hence you might to install some conda packages on your own if they are required by some commands in docker_file.)

Another option would be to install additional packages within the R or Python code (in case the container will be connected to the required network). However in that case you may end up in unnecessary latencies. So installing all dependencies once in the image should always be preferred.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

conda_dependencies = CondaDependencies.create(
    conda_packages=['rpy2', 'mro-base','r-jsonlite','r-essentials'])

# only needed if the model data collection feature is used
conda_dependencies.add_pip_package("azureml-monitoring")

with open("conda_dependencies.yml","w") as f:
    f.write(conda_dependencies.serialize_to_string())

Create the image (this might take a while).

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script="score.py",
    runtime="python",
    conda_file="conda_dependencies.yml",
    description=
    "Sample image to show the usage of R with Azure Machine Learning Services.",
    tags={
        'area': "samples",
        'type': "regression"
    },
)

image = ContainerImage.create(name="r-on-amls-sample",
                              models=[model],
                              image_config=image_config,
                              workspace=workspace)

image.wait_for_creation(show_output=True)

# Deployment

There is multiple engines where the image we created before could be deployed to. Select the one which is most appropriate for you.

## Option 1: Deploy to ACI

ACI is good for quick dev tests. It's not recommended for production workloads. For an SSL deployment to ACI, you need to bring your own SSL certificate (= different to AKS where Microsoft can provide certificates if desired). You can use the attached [PowerShell script](Create-SelfSignedCertificate.ps1) to create a self-signed certificate for inital testing purposes. The script will also install the certificate into the certificate store of your computer.

Note however that webservice clients might not work with self-signed certificates, and they are not appropriate for production workloads. **When creating a certificate, you have to ensure that the DNS names match. Otherwise, the certificate validation will fail later.** Also keep in mind that certificates expire (depending on what you configured. usually, certificates expire after one year).

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

## configuration
service_name = "r-on-aml-service-aci"
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={
        'area': "samples",
        'type': "regression"
    },
    description=
    'Sample image to show the usage of R with Azure Machine Learning Services.',
    auth_enabled=True,

    # note: - required if SSL is used, optional if SSL is not used
    #       - GIVE IT TO YOUR OWN NAME BEFORE DEPLOYING!!!
    #       - only lowercase letters, numbers and hyphens.
    #         first character must be a letter.
    #         last character must be a letter or number.
    #         value must be between 5 and 63 characters long.
    dns_name_label="fidge-my-new-aci-webservice",

    # SSL- comment/uncomment as needed
    ssl_enabled=True,
    ssl_cname="fidge-my-new-aci-webservice.westeurope.azurecontainer.io",
    ssl_cert_pem_file="certificate_for_amls.pem",
    ssl_key_pem_file="private_key_for_amls.pem")

## do deployment
aci_service = Webservice.deploy_from_image(workspace=workspace,
                                           name="r-on-aml-service-aci",
                                           deployment_config=deployment_config,
                                           image=image)
aci_service.wait_for_deployment(show_output=True)

# inform about URIs and auth
print("Scoring URI: {}".format(aci_service.scoring_uri))
print("Swagger URI: {}".format(aci_service.swagger_uri))
print("Auth Keys: {}".format(aci_service.get_keys()))

## Option 2: Deploy to existing AKS (Kubernetes)

AKS should be used for production workloads. Note that for AKS, SSL encryption is configured at the AKS cluster. Besides, the model data collection feature is only supported on AKS.

In [ ]:
from azureml.core import ComputeTarget
from azureml.core.webservice import AksWebservice, Webservice

aks_cluster_name = "aks-cluster"
aks_service_name = 'r-on-aml-service-aks'
collect_model_data = True
enable_app_insights = True

# deploy webservice
aks_service_list = [webservice for webservice in AksWebservice.list(workspace) if webservice.name == aks_service_name]
aks_service = aks_service_list[0] if len(aks_service_list) == 1 else None
if aks_service:   
    print("Updating existing service...")
    aks_service.update(image=image,
                       enable_app_insights=enable_app_insights,
                       collect_model_data=collect_model_data)
    print("Update completed.")

else:
    print("Deploying new webservice...")
    aks_service = Webservice.deploy_from_image(workspace=workspace,
                                                   name=aks_service_name,
                                                   image=image,
                                                   deployment_config=AksWebservice.deploy_configuration(
                                                       collect_model_data=collect_model_data,
                                                       enable_app_insights=enable_app_insights
                                                   ),
                                                   deployment_target=ComputeTarget(
                                                       workspace=workspace,
                                                       name=aks_cluster_name))
    aks_service.wait_for_deployment(show_output=True)
    print("Creation completed.")
print("")

print("Scoring URI: {}".format(aks_service.scoring_uri))
print("Swagger URI: {}".format(aks_service.swagger_uri))
print("Keys: {}".format(aks_service.get_keys()))

## Option 3: Deploy to Azure Web Apps for Containers

This option is currently in preview. See [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-app-service) for more details.

# Test the web service
To test the webservice, use the accompanying [Consume Webservice](consume-webservice.ipynb) notebook. Use the scoring URI and one of the keys printed above.

# Clean up
Don't forget to clean up the resources you created.

In [ ]:
#if aci_service:
#    aci_service.delete()
#if aks_service:
#    aks_service.delete()
#image.delete()
#model.delete()